## Scrapy 사용

In [ ]:
import scrapy
from NaverNews.items import NavernewsItem 
from datetime import datetime, timedelta
import pandas as pd

import time
import re

category = {
  #정치
  '100' : ['264','265','268','266','267', '269'],
  #세계
  '104' : ['231','232','233','234','322']
}

category_dict = {
    '100' : '정치',
    '104' : '세계',

    #정치
    '264' : '대통령실',
    '265' : '국회/정당',
    '268' : '북한',
    '266' : '행정',
    '267' : '국방/외교',
    '269' : '정치일반',

    #세계
    '231' : '아시아/호주',
    '232' : '미국/중남미',
    '233' : '유럽',
    '234' : '중동/아프리카',
    '322' : '세계일반'
    
}


class NewsSpider(scrapy.Spider):
    name = "news"
    allowed_domains = ["news.naver.com"]
    start_urls = ["http://news.naver.com/"]

    # Scrapy로 크롤링을 실행시키면 이 함수가 실행됩니다. 
    # 처음 시작하는 요청 url을 만들어 줌
    def start_requests(self):

        # 탐색할 날짜 리스트 생성
        date_range = pd.date_range(start='20230201', end='20230316')
        date_list = date_range.strftime("%Y%m%d").to_list()

        for date in date_list:
            for main, sub_cats in category.items(): 
                for sub_cat in sub_cats:
                    url = f'https://news.naver.com/main/list.naver?mode=LS2D&mid=shm&sid2={sub_cat}&sid1={main}&date={date}&page=1'
                    # meta는 다양한 정보를 보관하는 보관함
                    # yield는 scheduler에 request를 하나씩 넣어주는 역할
                    # response.meta를 통해 request에서 보낸 정보에 접근
                    ### yield scrapy.Request(url, callback, meta) ###
                    # callback: request를 수행한 후 실행할 함수
                    # meta: 추가 정보를 담는 딕셔너리 객체. 이후 callback함수에서 이 정보 활용
                    yield scrapy.Request(url, self.url_parse, meta = {'page' : 1, 'urls' : [],  'main' : main, 'sub' : sub_cat})

    # 각 뉴스기사의 url 가져오기
    def url_parse(self, response):
        # css 셀렉터
        urls = response.css('#main_content dt.photo a::attr(href)').getall()

        # 종료 조건. 끝나는 조건을 만족해 return으로 크롤링 종료
        # 중복된 url이 없으면 다음페이지 크롤링, 있다면 크롤링 중지
        # 이전 페이지에서 보관한 urls 목록을 가져와서 urls 변수와 비교
        # pop() 메서드는 urls키와 해당하는 값을 삭제하면서 반환
        if response.meta.pop('urls') == urls :
            return 
        
        # 기사 본문을 하나씩 가져오기
        for url in urls :
            yield scrapy.Request(url, self.parse_news, meta = response.meta)

        # 다음 페이지
        page = response.meta.pop('page')
        # 'page=숫자'일 때 'page=숫자+1'로 대체해 다음페이지 url 만들어줌
        # re.sub(pattern, replace text, string, count=0)
        next_page_url = re.sub('page\=\d+', f'page={page+1}', response.url)
        # page가 +1 되고, urls 빈 리스트에 이제는 현재 페이지의 뉴스기사가 리스트로 저장됨
        # page 1 증가시킨 사이트(다음페이지)에 요청
        yield scrapy.Request(next_page_url, self.url_parse, meta={**response.meta, 'page':page+1, 'urls': urls})


    # 뉴스기사 상세 정보 크롤링
    def parse_news(self, response):

        item = NavernewsItem()
        # 기사 제목
        item['title'] = response.css('#title_area.media_end_head_headline span::text').get()
        # 언론사명
        item['source'] = response.css('#ct a img::attr(title)').get()
        item['platform'] = '네이버'
        # 카테고리
        item['main_category'] = category_dict[response.meta.pop('main')]
        item['sub_category'] = category_dict[response.meta.pop('sub')]
        # 본문 내용
        sen_list = response.css('#dic_area::text').getall()
        item['content'] = ' '.join(sen_list).strip()
        # 작성 날짜
        item['writed_at'] = response.css('.media_end_head_info_datestamp_bunch span::attr(data-date-time)').getall()[0]
        # 기자 이름
        item['writer'] = response.css('.media_end_head_journalist_name::text').get()

        yield item

In [2]:
len(pd.read_csv('./NaverNews/NaverNews/spiders/NaverNews.csv'))

158267